In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("test.db")
print("SQLite connected.")


SQLite connected.


In [3]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("lendingclub.db")

# Only import the first 500 000 rows + essential columns
usecols = [
    "loan_amnt", "term", "int_rate", "installment", "grade", "sub_grade",
    "emp_length", "home_ownership", "annual_inc", "issue_d", "loan_status",
    "purpose", "addr_state", "dti", "delinq_2yrs", "revol_util", "total_acc"
]

df = pd.read_csv("accepted_2007_to_2018Q4.csv", usecols=usecols, nrows=500_000, low_memory=False)
print(df.shape)
df.head()


(500000, 17)


,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc
0,3600.0,36 months,13.99,123.03,C,C4,10+ years,MORTGAGE,55000.0,Dec-2015,Fully Paid,debt_consolidation,PA,5.91,0.0,29.7,13.0
1,24700.0,36 months,11.99,820.28,C,C1,10+ years,MORTGAGE,65000.0,Dec-2015,Fully Paid,small_business,SD,16.06,1.0,19.2,38.0
2,20000.0,60 months,10.78,432.66,B,B4,10+ years,MORTGAGE,63000.0,Dec-2015,Fully Paid,home_improvement,IL,10.78,0.0,56.2,18.0
3,35000.0,60 months,14.85,829.90,C,C5,10+ years,MORTGAGE,110000.0,Dec-2015,Current,debt_consolidation,NJ,17.06,0.0,11.6,17.0
4,10400.0,60 months,22.45,289.91,F,F1,3 years,MORTGAGE,104433.0,Dec-2015,Fully Paid,major_purchase,PA,25.37,1.0,64.5,35.0


In [4]:
# Write the dataframe into the SQLite database as a table named "loans"
df.to_sql("loans", conn, if_exists="replace", index=False)
# Confirm record count
pd.read_sql_query("SELECT COUNT(*) AS total_records FROM loans;", conn)


,total_records
0,500000


In [5]:
pd.read_sql_query("""
SELECT loan_status, COUNT(*) AS num_loans
FROM loans
GROUP BY loan_status
ORDER BY num_loans DESC;
""", conn)


,loan_status,num_loans
0,Fully Paid,312340
1,Current,104240
2,Charged Off,78824
3,Late (31-120 days),2977
4,In Grace Period,1046
5,Late (16-30 days),567
6,Default,4
7,None,2


In [6]:
pd.read_sql_query("""
SELECT 
    grade,
    COUNT(*) AS total_loans,
    SUM(CASE WHEN loan_status IN ('Charged Off','Default','Late (31-120 days)','Late (16-30 days)') THEN 1 ELSE 0 END) AS defaulted_loans,
    ROUND(SUM(CASE WHEN loan_status IN ('Charged Off','Default','Late (31-120 days)','Late (16-30 days)') THEN 1 ELSE 0 END)*100.0 / COUNT(*), 2) AS default_rate_percent
FROM loans
GROUP BY grade
ORDER BY grade;
""", conn)


,grade,total_loans,defaulted_loans,default_rate_percent
0,None,2,0,0.00
1,A,92870,4392,4.73
2,B,141336,15213,10.76
3,C,141680,25587,18.06
4,D,73872,18983,25.70
5,E,37663,12733,33.81
6,F,10317,4396,42.61
7,G,2260,1068,47.26


In [7]:
pd.read_sql_query("""
SELECT 
    grade,
    ROUND(AVG(int_rate), 2) AS avg_interest_rate
FROM loans
GROUP BY grade
ORDER BY grade;
""", conn)


,grade,avg_interest_rate
0,None,NaN
1,A,6.90
2,B,10.13
3,C,13.43
4,D,17.10
5,E,19.70
6,F,23.89
7,G,27.00


In [9]:
pd.read_sql_query("""
SELECT 
    grade,
    ROUND(AVG(dti), 2) AS avg_dti,
    ROUND(AVG(annual_inc), 0) AS avg_income
FROM loans
WHERE annual_inc IS NOT NULL AND dti IS NOT NULL
GROUP BY grade
ORDER BY grade;
""", conn)


,grade,avg_dti,avg_income
0,A,16.42,90542.0
1,B,18.24,79328.0
2,C,19.88,73076.0
3,D,21.69,68690.0
4,E,22.02,70719.0
5,F,21.83,71661.0
6,G,20.96,70104.0


In [10]:
#######Loan Volume and Default Rate by Year######
pd.read_sql_query("""
SELECT 
    SUBSTR(issue_d, INSTR(issue_d, '-') + 1) AS issue_year,
    COUNT(*) AS total_loans,
    SUM(CASE WHEN loan_status IN ('Charged Off','Default','Late (31-120 days)','Late (16-30 days)') THEN 1 ELSE 0 END) AS defaulted_loans,
    ROUND(SUM(CASE WHEN loan_status IN ('Charged Off','Default','Late (31-120 days)','Late (16-30 days)') THEN 1 ELSE 0 END)*100.0/COUNT(*), 2) AS default_rate_percent
FROM loans
GROUP BY issue_year
ORDER BY CAST(issue_year AS INTEGER);
""", conn)


,issue_year,total_loans,defaulted_loans,default_rate_percent
0,None,2,0,0.00
1,2015,421095,77442,18.39
2,2018,78903,4930,6.25


In [11]:
portfolio_summary = pd.read_sql_query("""
SELECT 
    grade,
    SUBSTR(issue_d, INSTR(issue_d, '-') + 1) AS issue_year,
    COUNT(*) AS total_loans,
    ROUND(AVG(int_rate), 2) AS avg_interest_rate,
    ROUND(AVG(dti), 2) AS avg_dti,
    ROUND(AVG(annual_inc), 0) AS avg_income,
    SUM(CASE WHEN loan_status IN ('Charged Off','Default','Late (31-120 days)','Late (16-30 days)') THEN 1 ELSE 0 END) AS defaulted_loans,
    ROUND(SUM(CASE WHEN loan_status IN ('Charged Off','Default','Late (31-120 days)','Late (16-30 days)') THEN 1 ELSE 0 END)*100.0/COUNT(*), 2) AS default_rate_percent
FROM loans
WHERE annual_inc IS NOT NULL AND dti IS NOT NULL
GROUP BY grade, issue_year
ORDER BY CAST(issue_year AS INTEGER), grade;
""", conn)

portfolio_summary.head()


,grade,issue_year,total_loans,avg_interest_rate,avg_dti,avg_income,defaulted_loans,default_rate_percent
0,A,2015,73336,6.94,16.28,91316.0,4007,5.46
1,B,2015,117606,10.04,18.05,79193.0,14180,12.06
2,C,2015,120566,13.30,19.73,72800.0,24001,19.91
3,D,2015,62653,16.72,21.54,68464.0,17630,28.14
4,E,2015,34948,19.29,21.82,70850.0,12285,35.15


In [12]:
portfolio_summary.to_csv("portfolio_summary.csv", index=False)
print("Exported portfolio_summary.csv")


Exported portfolio_summary.csv
